<a href="https://colab.research.google.com/github/hasancsesu/Civil_Engineering_LLM_Assistant/blob/main/CivilEnggQ%26Aassistant_fine_tuned_by_Mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab Cell 1: Installation
# Installing Hugging Face ecosystem libraries: transformers, datasets, peft (for LoRA)
# trl (for SFTTrainer), accelerate, and the crucial bitsandbytes for QLoRA
!pip install -q transformers peft accelerate trl bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.5 MB/s eta 0:00:00


In [ ]:
# Colab Cell 2: Configuration and Login
import torch
from datasets import Dataset
from huggingface_hub import notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 1. Define Model and Directories
model_id = "mistralai/Mistral-7B-v0.1"
output_dir = "./mistral_civil_engineering_adapter"

# 2. Log in to Hugging Face
# You don't strictly need this for the open Mistral model, but it's good practice.
notebook_login()

# 3. Define 4-bit Quantization Configuration (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 # Use bfloat16 for computation
)

# 4. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Mistral uses the EOS token for padding
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Ensure padding is on the right for training

# 5. Load Model with QLoRA
print("Loading Model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
print("Model Loaded Successfully.")

# 🛑 CRUCIAL FIX A: Preparing the model for QLoRA/kbit training.
# This casts the layernorms to FP32 and ensures all required modules are trainable.
from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)

# Enabling gradient checkpointing for memory efficiency (crucial for T4)
# Note: This is now enabled *after* preparing the model for kbit training.
model.gradient_checkpointing_enable()

In [ ]:
# Colab Cell 3: Data Loading and Formatting

import pandas as pd
from datasets import Dataset

# The data is stored in the 'example' variable as per my format:
example = [
    {
        "instruction": "What is bearing capacity of soil?",
        "input": "",
        "output": "Bearing capacity is the maximum load per unit area which the soil can carry safely without shear failure and without excessive settlement. It is a key concept in foundation design."
    },
    {
        "instruction": "Explain the difference between dead loads and live loads in structural engineering.",
        "input": "",
        "output": "Dead loads are permanent, static loads such as self-weight of structural elements, finishes, and fixed equipment. Live loads are variable or moving loads such as occupants, furniture, vehicles, and stored materials, which may change over time."
    },
    {
        "instruction": "What is the purpose of reinforcement in reinforced concrete?",
        "input": "",
        "output": "Reinforcement, usually steel bars or meshes, is provided in concrete to resist tensile and sometimes compressive stresses. Concrete is strong in compression but weak in tension, so reinforcement allows the composite material to safely carry bending and tensile forces."
    },
    {
        "instruction": "Briefly explain the difference between working stress design and limit state design.",
        "input": "",
        "output": "Working stress design is based on elastic behavior and uses a factor of safety applied to permissible stresses. Limit state design considers ultimate strength and serviceability limit states separately and applies partial safety factors to loads and material strengths, providing a more rational and economical design."
    },
    {
        "instruction": "What is a shear failure in soil, and why is it important in foundation design?",
        "input": "",
        "output": "Shear failure in soil occurs when the stresses induced by loads exceed the shear strength of the soil, causing a slip or failure surface. It is crucial in foundation design because exceeding the soil’s shear strength can lead to sudden and catastrophic settlement or tilting of structures."
    },
    {
        "instruction": "Define slump test and its significance in concrete work.",
        "input": "",
        "output": "The slump test is a simple field test used to measure the consistency and workability of fresh concrete. A standard cone is filled with concrete, lifted, and the decrease in height (slump) is measured. Proper slump ensures the concrete is neither too stiff nor too fluid for placement and compaction."
    },
    {
        "instruction": "What is the difference between flexible pavement and rigid pavement?",
        "input": "",
        "output": "Flexible pavement, usually made of asphalt layers, distributes loads through a layered system and deforms under traffic. Rigid pavement, made of concrete slabs, has high flexural strength and distributes loads over a wider area with relatively little deformation. Flexible pavements are more sensitive to subgrade quality, while rigid pavements are more durable but costlier."
    },
    {
        "instruction": "Explain freeboard in the context of hydraulic structures.",
        "input": "",
        "output": "Freeboard is the vertical distance between the maximum design water level and the top of a channel, dam, or embankment. It provides a safety margin against wave action, settlement, and unforeseen increases in water level."
    },
    {
        "instruction": "What is factor of safety in geotechnical engineering?",
        "input": "",
        "output": "Factor of safety is the ratio of the ultimate capacity (e.g., ultimate bearing capacity or resisting moment) to the actual working load or effect. It accounts for uncertainties in soil properties, loading, and analysis methods, ensuring a margin of safety against failure."
    },
    {
        "instruction": "As a civil engineer, how would you briefly advise a client about controlling cracks in reinforced concrete slabs?",
        "input": "",
        "output": "Crack control in reinforced concrete slabs involves proper detailing of reinforcement (adequate bar spacing and cover), using appropriate concrete mix with controlled water–cement ratio, proper curing, and avoiding restraint conditions and rapid temperature or moisture changes. Good construction practices and joint detailing also help limit crack width."
    },
    {
        "instruction": "What is the ultimate bearing capacity of soil?",
        "input": "",
        "output": "The ultimate bearing capacity is the maximum pressure a soil can sustain just before shear failure occurs. It represents the theoretical upper limit of stress that can be applied to the soil under a foundation."
    },
    {
        "instruction": "List the primary factors that influence the bearing capacity of soil.",
        "input": "",
        "output": "Key factors include soil type, density, shear strength parameters (cohesion and angle of internal friction), groundwater conditions, footing shape and size, depth of foundation, and load inclination."
    },
    {
        "instruction": "What are the types of shear failure in soils?",
        "input": "",
        "output": "The three primary types are: 1. General shear failure – distinct failure surface, sudden collapse. 2. Local shear failure – partial failure, no well-defined surface. and, 3. Punching shear failure – occurs in loose or soft soils under heavily loaded small foundations."
    },
    {
        "instruction": "What is swelling soil?",
        "input": "",
        "output": "Swelling soils, also called expansive soils, contain minerals such as montmorillonite that absorb water and expand significantly, causing uplift, cracking, and structural damage."
    },
    {
        "instruction": "What is consolidation in soil mechanics?",
        "input": "",
        "output": "Consolidation is the gradual reduction in soil volume caused by expulsion of pore water under sustained load. It governs long-term settlement in clayey soils."
    },
    {
        "instruction": "How is safe bearing capacity defined?",
        "input": "",
        "output": "Safe bearing capacity is the maximum pressure that can be safely applied to the soil without causing shear failure and while keeping settlement within acceptable limits. It is computed by dividing ultimate capacity by a factor of safety."
    },
    {
        "instruction": "Differentiate between compaction and consolidation.",
        "input": "",
        "output": "Compaction is the reduction in soil volume by expelling air through mechanical effort (instant). Consolidation is the reduction of volume by water expulsion under load (time-dependent)."
    },
    {
        "instruction": "What does the angle of internal friction represent?",
        "input": "",
        "output": "It represents the shear resistance provided by soil due to interparticle friction. Sands have high friction angles, while clays typically have low values."
    },
    {
        "instruction": "Differentiate between shallow and deep foundations.",
        "input": "",
        "output": "Shallow foundations transmit loads to the soil at relatively small depths (1–3m). Examples: isolated footings, raft foundations. Deep foundations transfer loads to deeper, stronger layers (10–60m). Examples: piles, drilled shafts."
    },
    {
        "instruction": "List common types of piles used in civil engineering.",
        "input": "",
        "output": "Common types include friction piles, end-bearing piles, composite piles, timber piles, steel H-piles, precast concrete piles, and bored cast-in-situ piles."
    },
    {
        "instruction": "What is negative skin friction on piles?",
        "input": "",
        "output": "Negative skin friction occurs when surrounding soil settles more than the pile, dragging the pile downward and adding additional load. It often happens in soft clays or areas of soil consolidation."
    },
    {
        "instruction": "When is a raft foundation preferred?",
        "input": "",
        "output": "Rafts are used when soil is weak, differential settlement must be minimized, or building columns are closely spaced. They provide uniform load distribution."
    },
    {
        "instruction": "What is a pile cap?",
        "input": "",
        "output": "A pile cap is a reinforced concrete block that ties multiple piles together and transfers structural loads to the pile group."
    },
    {
        "instruction": "What are the types of settlement in foundations?",
        "input": "",
        "output": "Immediate settlement (elastic), consolidation settlement (time-dependent), and secondary compression settlement."
    },
    {
        "instruction": "instruction: What is reinforced concrete?",
        "input": "",
        "output": "Reinforced concrete is concrete that contains steel bars or mesh to resist tension and improve overall strength."
    },
    {
        "instruction": "instruction: What is cover in RCC?",
        "input": "",
        "output": "Cover is the minimum thickness of concrete over reinforcement to protect steel from corrosion and fire."
    },
    {
        "instruction": "Define slump",
        "input": "",
        "output": "Slump is a simple field test that measures the workability of fresh concrete."
    },
    {
        "instruction": "What causes cracks in concrete?",
        "input": "",
        "output": "Cracks can be caused by shrinkage, temperature changes, poor curing, excessive loading, or insufficient reinforcement."
    },
    {
        "instruction": "What is curing?",
        "input": "",
        "output": "Curing is the process of keeping concrete moist to ensure proper strength development."
    },
    {
        "instruction": "What is a one-way slab?",
        "input": "",
        "output": "A one-way slab mainly bends in one direction because it is supported on two opposite sides."
    },
    {
        "instruction": " Why is steel used in buildings?",
        "input": "",
        "output": ": Steel is strong, durable, and can carry large loads with relatively small sections."
    },
    {
        "instruction": " What is structural steel?",
        "input": "",
        "output": ": Structural steel is steel specifically shaped and treated for use in building frameworks."
    },
    {
        "instruction": " What is buckling?",
        "input": "",
        "output": ": Buckling is the sudden sideways failure of a structural member under compressive load."
    },
    {
        "instruction": " What is a truss?",
        "input": "",
        "output": ": A truss is a framework of connected members arranged in triangles to efficiently carry loads."
    },
    {
        "instruction": " What is corrosion?",
        "input": "",
        "output": ": Corrosion is the chemical deterioration of steel due to moisture and oxygen."
    },
    {
        "instruction": " What is a steel column?",
        "input": "",
        "output": ": A steel column is a vertical member designed to carry compressive loads."
    },
    {
        "instruction": " What is welding?",
        "input": "",
        "output": ": Welding is the process of joining metal parts by melting and fusing them."
    },
    {
        "instruction": "What is a flexible pavement?",
        "input": "",
        "output": " Flexible pavement is made of asphalt layers and distributes load through gradual deformation."
    },
    {
        "instruction": "What is a rigid pavement?",
        "input": "",
        "output": " Rigid pavement uses concrete slabs that distribute load over a wide area with minimal deformation."
    },
    {
        "instruction": "What is a camber?",
        "input": "",
        "output": "Camber is the upward slope of a road surface to drain rainwater."
    },
    {
        "instruction": "What is traffic volume?",
        "input": "",
        "output": " Traffic volume is the number of vehicles passing a point in a specified time."
    },
    {
        "instruction": "What is a median?",
        "input": "",
        "output": " A median is a divider between opposite lanes to improve safety."
    },
    {
        "instruction": "What is a super-elevation?",
        "input": "",
        "output": " Super-elevation is the raising of the outer edge of a road on a curve to counter vehicle overturning."
        },
    {
        "instruction": "What is grade?",
        "input": "",
        "output": " Grade is the slope of a road measured as a percentage."
    },
    {
        "instruction": "What is a pothole?",
        "input": "",
        "output": "A pothole is a depression in the road surface caused by water and repeated traffic loads."
    },
    {
        "instruction": "What is bitumen?",
        "input": "",
        "output": " Bitumen is a black, sticky binder used in asphalt pavement. "
    },
    {
        "instruction": "What is discharge?",
        "input": "",
        "output": "Discharge is the volume of water flowing through a channel per unit time."
    },
    {
        "instruction": "What is a weir?",
        "input": "",
        "output": "A weir is a small barrier across a river used to measure or control flow."
    },
    {
        "instruction": "What is freeboard?",
        "input": "",
        "output": "Freeboard is the vertical gap between water level and the top of a structure."
    },
    {
        "instruction": "What is runoff?",
        "input": "",
        "output": "Runoff is the portion of rainfall that flows over the ground surface."
    },
    {
        "instruction": "What is infiltration?",
        "input": "",
        "output": "Infiltration is the process of water entering the soil."
    },
    {
        "instruction": "What is a channel?",
        "input": "",
        "output": "A channel is a natural or artificial path through which water flows."
    },
    {
        "instruction": "What is erosion?",
        "input": "",
        "output": "Erosion is the removal of soil by flowing water or wind."
    },
    {
        "instruction": "What is a hydrograph?",
        "input": "",
        "output": "A hydrograph is a graph that shows how river flow varies with time."
    },
    {
        "instruction": "What is a project schedule?",
        "input": "",
        "output": "A project schedule shows the sequence and timing of construction activities."
    },
    {
        "instruction": "What is cost estimation?",
        "input": "",
        "output": "Cost estimation predicts the expenses required for a construction project."
    },
    {
        "instruction": "What is quality control?",
        "input": "",
        "output": "Quality control ensures materials and work meet required standards."
    },
    {
        "instruction": "instruction: What is safety management?",
        "input": "",
        "output": "Safety management focuses on preventing accidents on construction sites."
    },
    {
        "instruction": "What is procurement?",
        "input": "",
        "output": "Procurement is the process of purchasing materials and services for a project."
    },
    {
        "instruction": "What is a contract?",
        "input": "",
        "output": "A contract is a legal agreement defining responsibilities and obligations."
    },
    {
        "instruction": "What is a BOQ?",
        "input": "",
        "output": "A BOQ is a Bill of Quantities listing materials and their amounts."
    },
    {
        "instruction": "What is wastewater?",
        "input": "",
        "output": "Wastewater is used water containing impurities from homes or industries."
    },
    {
        "instruction": "What is a septic tank?",
        "input": "",
        "output": "A septic tank is an underground chamber that treats household wastewater."
    },
    {
        "instruction": "What is water treatment?",
        "input": "",
        "output": "Water treatment removes contaminants to make water safe for use."
    },
    {
        "instruction": "What is solid waste?",
        "input": "",
        "output": "Solid waste is any garbage or refuse produced by human activity."
    },
    {
        "instruction": "What is air pollution?",
        "input": "",
        "output": "Air pollution occurs when harmful substances enter the atmosphere."
    },
    {
        "instruction": "What is chlorination?",
        "input": "",
        "output": "BOD is the amount of oxygen required by microorganisms to break down organic matter."
    },
    {
        "instruction": "What is a load?",
        "input": "",
        "output": "A load is any external force applied to a structure."
    },
    {
        "instruction": "What is bending moment?",
        "input": "",
        "output": "Bending moment is the internal force that causes a beam to bend."
    },
    {
        "instruction": "What is shear force?",
        "input": "",
        "output": "Shear force is the force that acts parallel to the cross-section of a beam."
    },
    {
        "instruction": "What is a simply supported beam?",
        "input": "",
        "output": "A simply supported beam is supported at both ends but free to rotate."
    },
    {
        "instruction": "What is a fixed support?",
        "input": "",
        "output": "A fixed support restricts all movements of a structural member."
    },
    {
        "instruction": "What is deflection?",
        "input": "",
        "output": "Deflection is the vertical displacement of a structural element under load."
    },
    {
        "instruction": "What is stiffness?",
        "input": "",
        "output": "Stiffness is a structure’s resistance to deformation."
    },
    {
        "instruction": "What is a frame?",
        "input": "",
        "output": "A frame is a structure with beams and columns connected together."
    },
    {
        "instruction": "What is leveling?",
        "input": "",
        "output": "Leveling is the process of determining height differences between points."
    },
    {
        "instruction": "What is a benchmark?",
        "input": "",
        "output": "A benchmark is a known elevation point used as a reference."
    },
    {
        "instruction": "What is a total station?",
        "input": "",
        "output": "A total station is an electronic device used to measure angles and distances."
    },
    {
        "instruction": "What is GPS in surveying?",
        "input": "",
        "output": "GPS is a satellite-based system used to determine precise locations."
    },
    {
        "instruction": "What is contouring?",
        "input": "",
        "output": "Contouring involves connecting points of equal elevation on a map."
    },
    {
        "instruction": "What is a traverse?",
        "input": "",
        "output": "A traverse is a series of connected survey lines whose lengths and angles are measured."
    },
    {
        "instruction": "What is EDM?",
        "input": "",
        "output": "EDM stands for Electronic Distance Measurement used for accurate distance readings."
    },
    {
        "instruction": "What is a theodolite?",
        "input": "",
        "output": "A theodolite is an instrument used to measure horizontal and vertical angles."
    }
]


# 1. Convert JSON list to Hugging Face Dataset
df = pd.DataFrame(example)
dataset = Dataset.from_pandas(df)

# 2. Define the formatting function to match the instruction template
def format_data(example):
    # We ignore the 'input' field as it's empty, focusing on Instruction and Output
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    }

# Applying the formatting
formatted_dataset = dataset.map(format_data, remove_columns=["instruction", "input", "output"])

# 3. Splitting into training and test sets
# We use a 90/10 split (90% for training, 10% for evaluation)
processed_dataset = formatted_dataset.train_test_split(test_size=0.1)

train_dataset = processed_dataset["train"]
eval_dataset = processed_dataset["test"]

print(f"Total entries: {len(formatted_dataset)}")
print(f"Training set size: {len(train_dataset)}")
print(f"Evaluation set size: {len(eval_dataset)}")
print("-" * 30)
print("Example of formatted data:")
print(train_dataset[0]['text'])

In [ ]:
# Colab Cell 4: LoRA and Training Setup

from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer
# Note: model and tokenizer were loaded in Cell 2

# 1. LoRA Configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    #CRUCIAL FIX B: Ensure all required linear layers are targeted for Mistral
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# 2. Apply LoRA to the QLoRA Model
model = get_peft_model(model, lora_config)

# Check trainable parameters (should be a tiny percentage)
model.print_trainable_parameters()


# 3. Training Arguments (T4 Optimization is KEY here)
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3, # Recommended 3-5 epochs for SFT
    per_device_train_batch_size=1, # **Crucial for T4 VRAM**
    gradient_accumulation_steps=4, # Simulates a batch size of 4 (1 * 4)
    optim="paged_adamw_8bit", # Use paged 8-bit AdamW for optimizer memory savings
    learning_rate=2e-4, # Standard LLM fine-tuning rate
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True, # Enable 16-bit precision for faster computation
    max_grad_norm=0.3, # Helps stabilize training
    warmup_ratio=0.03,
    load_best_model_at_end=True,
    report_to="none" # Disable logging to external platforms if not needed
)

# 4. SFT Trainer Setup
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    # 🛑 THE FIX: REMOVE the argument entirely, as the column name 'text'
    # is the default the SFTTrainer is looking for when the argument is missing.
    processing_class=tokenizer,
    args=training_args,
    #max_seq_length=1024, # Max token length for input sequence
)

In [ ]:
# Colab Cell 5: Start Training and Save Adapter

print("Starting Fine-Tuning...")
print("This process will take several hours, depending on your dataset size and Colab GPU.")

# Start Fine-Tuning
# The 'trainer' object was configured and initialized successfully in Cell 4
trainer.train()

# --- Saving the Fine-Tuned Model and Tokenizer ---

# Saving the final LoRA adapter weights (the small, trained part)
# This saves the adapter weights into the specified directory
trainer.model.save_pretrained(output_dir)

# Saving the tokenizer (required for loading and inference)
tokenizer.save_pretrained(output_dir)

print("\n🎉 Fine-tuning Complete!")
print(f"LoRA Adapter weights and tokenizer saved to: {output_dir}")
print("You can now proceed to load and test your Civil Engineering AI Assistant.")

In [ ]:
# Colab Cell 6: Inference and Testing

from peft import PeftModel
from transformers import pipeline

# 1. Reloading the base Mistral model (using the same 4-bit config)
print("Reloading base model and tokenizer...")
# We must use the same bnb_config from Cell 2 here
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model_reloaded = AutoModelForCausalLM.from_pretrained(
    model_id, # mistralai/Mistral-7B-v0.1
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

# 2. Load the LoRA adapter weights onto the base model
model_to_test = PeftModel.from_pretrained(
    base_model_reloaded,
    output_dir, # Loads weights from './mistral_civil_engineering_adapter'
)
model_to_test = model_to_test.merge_and_unload() # Optional: Merge adapter for better inference

# 3. Create a text generation pipeline
generator = pipeline(
    "text-generation",
    model=model_to_test,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# 4. Define a Civil Engineering Question
# Use the exact format you used for training!
prompt = "### Instruction:\nWhat are the main causes and remedies for differential settlement in structures?\n\n### Response:"

# 5. Generate the response
print("\n--- Generating Response ---")
result = generator(
    prompt,
    max_new_tokens=512, # Allow for a long, detailed response
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)

# Print the generated text
generated_text = result[0]['generated_text']
print(generated_text)

In [ ]:
## Colab Cell X: Running Test Queries

def run_test_query(question, generator):
    # Format the question using the exact training template
    prompt = f"### Instruction:\n{question}\n\n### Response:"

    print("-" * 60)
    print(f"QUERY: {question}")

    # Generate the response
    result = generator(
        prompt,
        max_new_tokens=512, # Keep this high for detailed answers
        do_sample=True,
        temperature=0.6, # Keep temperature lower (0.5-0.7) for factual accuracy
        top_p=0.9,
        eos_token_id=generator.tokenizer.eos_token_id # Use the tokenizer from the generator
    )

    # Extract and clean the generated text
    generated_text = result[0]['generated_text']

    # Simple cleaning: remove the input prompt part
    response = generated_text.replace(prompt, '').strip()

    print(f"\nASSISTANT RESPONSE:\n{response}")
    print("-" * 60)
    return response

# Ensure you have your 'generator' pipeline object from Cell 6
# (If you restarted the runtime, you must re-run Cell 6 to load the generator)

# --- Run Your Test Queries ---
test_questions = [
    "What is contouring in the context of surveying?",
    "What things a construction manager should remain aware of during praparation of a contract for procurement purpose?",
    "What are the best ways of treatment of municipal wastewater in a hot desert country like Morocco?",
]

for q in test_questions:
    run_test_query(q, generator)